In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report



[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=12702261a62771fa3f00aa280fa28e0ab3219c0c5e72925aa68f5c05d9091d73
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read Data

In [ ]:
 #read the csv file:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ANLP/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', error_bad_lines=False)

<ipython-input-34-c1effc4c212c>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ANLP/amazon_reviews_us_Office_Products_v1_00.tsv.gz', sep='\t', error_bad_lines=False)
Skipping line 20773: expected 15 fields, saw 22
Skipping line 39834: expected 15 fields, saw 22
Skipping line 52957: expected 15 fields, saw 22
Skipping line 54540: expected 15 fields, saw 22

Skipping line 80276: expected 15 fields, saw 22
Skipping line 96168: expected 15 fields, saw 22
Skipping line 96866: expected 15 fields, saw 22
Skipping line 98175: expected 15 fields, saw 22
Skipping line 112539: expected 15 fields, saw 22
Skipping line 119377: expected 15 fields, saw 22
Skipping line 120065: expected 15 fields, saw 22
Skipping line 124703: expected 15 fields, saw 22

Skipping line 134024: expected 15 fields, saw 22
Skipping line 153938: expected 15 fields, saw 22


## Keep Reviews and Ratings

In [ ]:
 #take only the features that are needed : review body and star rating
df = df[['review_body', 'star_rating']]

NameError: ignored

In [ ]:
df

,review_body,star_rating
0,Great product.,5
1,What's to say about this commodity item except...,5
2,"Haven't used yet, but I am sure I will like it.",5
3,Although this was labeled as &#34;new&#34; the...,1
4,Gorgeous colors and easy to use,4
...,...,...
2640249,I can't live anymore whithout my Palm III. But...,4
2640250,Although the Palm Pilot is thin and compact it...,4
2640251,This book had a lot of great content without b...,4
2640252,I am teaching a course in Excel and am using t...,5


# Data Cleaning



In [ ]:
#dropping empty rows
df.dropna(subset=['review_body', 'star_rating'], inplace=True)

avg_before_cleaning = df['review_body'].apply(len).mean()

In [ ]:
#getting the labels wrt star_rating
df['sentiment'] = df['star_rating'].apply(lambda rating: '1' if int(rating) >= 4 else '0')


In [ ]:
df

,review_body,star_rating,sentiment
0,Great product.,5,1
1,What's to say about this commodity item except...,5,1
2,"Haven't used yet, but I am sure I will like it.",5,1
3,Although this was labeled as &#34;new&#34; the...,1,0
4,Gorgeous colors and easy to use,4,1
...,...,...,...
2640249,I can't live anymore whithout my Palm III. But...,4,1
2640250,Although the Palm Pilot is thin and compact it...,4,1
2640251,This book had a lot of great content without b...,4,1
2640252,I am teaching a course in Excel and am using t...,5,1


In [ ]:
#to remove contracted words
contractions = {
    "ain't": "am not / is not / are not / has not / have not",
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "how'd": "how did",
    "how'll": "how will",
    "how's": "how is / how has / how does",
    "I'd": "I had / I would",
    "I'll": "I will",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'll": "it will",
    "it's": "it is / it has",
    "let's": "let us",
    "mightn't": "might not",
    "mustn't": "must not",
    "shan't": "shall not",
    "she'd": "she had / she would",
    "she'll": "she will",
    "shouldn't": "should not",
    "that's": "that is / that has",
    "there's": "there is / there has",
    "they'd": "they had / they would",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'll": "we will",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what're": "what are",
    "what's": "what is / what has",
    "what've": "what have",
    "who'd": "who had / who would",
    "won't": "will not",
    "wouldn't": "would not",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "aren't": "are not",
    "couldn't've": "could not have",
    "shouldn't've": "should not have",
    "wouldn't've": "would not have",
    "mightn't've": "might not have",
    "mustn't've": "must not have",
    "should've": "should have",
    "could've": "could have",
    "would've": "would have",
    "might've": "might have",
    "must've": "must have",
    "when's": "when is",
    "where'll": "where will",
    "where's": "where is",
    "who'll": "who will",
    "who're": "who are",
    "who's": "who is",
    "who've": "who have",
    "why'll": "why will",
    "why's": "why is",
    "won't": "will not",
    "wouldn't": "would not",
    "you'd": "you did",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have",
    "that'd": "that would",
    "that'd've": "that would have",
    "there'd": "there had",
    }

def expand_contractions(text, contractions):
    for contraction, full_form in contractions.items():
        text = re.sub(r'\b' + re.escape(contraction) + r'\b', full_form, text)
    return text

df['review_body'] = df['review_body'].apply(expand_contractions, args=(contractions,))


In [ ]:
df

,review_body,star_rating,sentiment
0,Great product.,5,1
1,What's to say about this commodity item except...,5,1
2,"Haven't used yet, but I am sure I will like it.",5,1
3,Although this was labeled as &#34;new&#34; the...,1,0
4,Gorgeous colors and easy to use,4,1
...,...,...,...
2640249,I cannot live anymore whithout my Palm III. Bu...,4,1
2640250,Although the Palm Pilot is thin and compact it...,4,1
2640251,This book had a lot of great content without b...,4,1
2640252,I am teaching a course in Excel and am using t...,5,1


In [ ]:
# cleaning all the reviews
def cleanReviews(text):
    # print(text)
    if pd.isna(text):
        return ""

    text = text.lower()
    text = text.strip() #to remove any trailing and leading spaces in the review
    text = re.sub(r'<.*?>', '', text) # to remove HTML tags
    text = re.sub(r'[^a-zA-Z\s]', '', text) # to remove special characters, punctuation, and symbols
    text = text.translate(str.maketrans('', '', string.punctuation)) #to remove punctuation

    return text




In [ ]:
df['review_body'] = df['review_body'].apply(cleanReviews) # applying clenaning to all the reviews where html tags and special char and punctuations are removed

In [ ]:
df

,review_body,star_rating,sentiment
0,great product,5,1
1,whats to say about this commodity item except ...,5,1
2,havent used yet but i am sure i will like it,5,1
3,although this was labeled as new the one i rec...,1,0
4,gorgeous colors and easy to use,4,1
...,...,...,...
2640249,i cannot live anymore whithout my palm iii but...,4,1
2640250,although the palm pilot is thin and compact it...,4,1
2640251,this book had a lot of great content without b...,4,1
2640252,i am teaching a course in excel and am using t...,5,1


In [ ]:
#to remove any empty reviews that are present after cleaning
df = df[df['review_body'] != '']


In [ ]:
#len of data after cleaning
avg_len_after_cleaning = df['review_body'].apply(len).mean()

print(f"{avg_before_cleaning:.2f}, {avg_len_after_cleaning:.2f}")


285.26, 274.24


 ## We form two classes and select 50000 reviews randomly from each class.



In [ ]:
#sampling the data and getting 50000 of positive and negetive data
positive_count = (df['sentiment'] == str(1)).sum()
negative_count = (df['sentiment'] == str(0)).sum()
num_samples = min(50000, positive_count, negative_count)

#sample the rows for each sentiment value
positive_samples = df[df['sentiment'] == str(1)].sample(n=num_samples, random_state=11)
negative_samples = df[df['sentiment'] == str(0)].sample(n=num_samples, random_state=11)


In [ ]:
#combine the sampled rows into a single DataFrame
sampled_df = pd.concat([positive_samples, negative_samples])

#index of the combined DataFrame is reset
sampled_df.reset_index(drop=True, inplace=True)
sdf = sampled_df


# Pre-processing

In [ ]:
#starting the preprocessing
avg_before_preprocessing = sdf['review_body'].apply(len).mean()



In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df

,review_body,star_rating,sentiment
0,great product,5,1
1,whats to say about this commodity item except ...,5,1
2,havent used yet but i am sure i will like it,5,1
3,although this was labeled as new the one i rec...,1,0
4,gorgeous colors and easy to use,4,1
...,...,...,...
2640249,i cannot live anymore whithout my palm iii but...,4,1
2640250,although the palm pilot is thin and compact it...,4,1
2640251,this book had a lot of great content without b...,4,1
2640252,i am teaching a course in excel and am using t...,5,1


## remove the stop words

In [ ]:
#removing stopwords from the reviews
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
sdf['filtered_review_body_wo_stopwords'] = sdf['tokenized_review_body'].apply(lambda tokens: [word for word in tokens if word not in stop_words])



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## perform lemmatization  

In [ ]:
sdf

,review_body,star_rating,sentiment,tokenized_rev,tokenized_review_body,filtered_review_body_wo_stopwords
0,very cool this was for my daughter who had her...,5,1,"[very, cool, this, was, for, my, daughter, who...","[very, cool, this, was, for, my, daughter, who...","[cool, daughter, share, needle, staying, hospi..."
1,it is so hard to find a good qualified binder ...,5,1,"[it, is, so, hard, to, find, a, good, qualifie...","[it, is, so, hard, to, find, a, good, qualifie...","[hard, find, good, qualified, binder, cabinet,..."
2,i love my rhinestone badge holder i get so man...,4,1,"[i, love, my, rhinestone, badge, holder, i, ge...","[i, love, my, rhinestone, badge, holder, i, ge...","[love, rhinestone, badge, holder, get, many, c..."
3,love it,5,1,"[love, it]","[love, it]",[love]
4,i use these folders to keep student files i lo...,5,1,"[i, use, these, folders, to, keep, student, fi...","[i, use, these, folders, to, keep, student, fi...","[use, folders, keep, student, files, love, fol..."
...,...,...,...,...,...,...
99995,two of these plush toys fell apart on christma...,1,0,"[two, of, these, plush, toys, fell, apart, on,...","[two, of, these, plush, toys, fell, apart, on,...","[two, plush, toys, fell, apart, christmas, mor..."
99996,it fits some of my equipment i would have like...,3,0,"[it, fits, some, of, my, equipment, i, would, ...","[it, fits, some, of, my, equipment, i, would, ...","[fits, equipment, would, liked, bigger, large,..."
99997,would be perfect if the bottom ring actually m...,2,0,"[would, be, perfect, if, the, bottom, ring, ac...","[would, be, perfect, if, the, bottom, ring, ac...","[would, perfect, bottom, ring, actually, met, ..."
99998,ok,3,0,[ok],[ok],[ok]


In [ ]:
#performing lemmatizaion
#doing pos tagging and then using it for lemmatization as it would work better for the nltk lemmatization
def get_pos(tag):
    if tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return nltk.corpus.wordnet.NOUN

nltk.download('averaged_perceptron_tagger')



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
df['pos_tags'] = sdf['filtered_review_body_wo_stopwords'].apply(pos_tag)



<ipython-input-57-5e2023eee11c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pos_tags'] = sdf['filtered_review_body_wo_stopwords'].apply(pos_tag)


In [ ]:
df

,review_body,star_rating,sentiment,pos_tags
0,great product,5,1,"[(cool, NN), (daughter, NN), (share, NN), (nee..."
1,whats to say about this commodity item except ...,5,1,"[(hard, JJ), (find, VBP), (good, JJ), (qualifi..."
2,havent used yet but i am sure i will like it,5,1,"[(love, VB), (rhinestone, NN), (badge, NN), (h..."
3,although this was labeled as new the one i rec...,1,0,"[(love, NN)]"
4,gorgeous colors and easy to use,4,1,"[(use, NN), (folders, NNS), (keep, VBP), (stud..."
...,...,...,...,...
2640249,i cannot live anymore whithout my palm iii but...,4,1,NaN
2640250,although the palm pilot is thin and compact it...,4,1,NaN
2640251,this book had a lot of great content without b...,4,1,NaN
2640252,i am teaching a course in excel and am using t...,5,1,NaN


In [ ]:
lemmatizer = WordNetLemmatizer()
sdf['lemmatized_review'] = sdf['pos_tags'].apply(lambda tags: [lemmatizer.lemmatize(word, get_pos(tag)) for word, tag in tags])


KeyError: ignored

In [ ]:

avg_len_after_preprocessing = sdf['lemmatized_review'].apply(len).mean()

df=sdf

df['review_body'] = df['lemmatized_review']

In [ ]:
avg_len_after_preprocessing = sdf['lemmatized_review'].apply(len).mean()

print(f"{avg_before_preprocessing:.2f}, {avg_len_after_preprocessing:.2f}")

301.38, 27.91


# TF-IDF and BoW Feature Extraction

In [ ]:
df.head()

,review_body,star_rating,sentiment,tokenized_review_body,filtered_review_body_wo_stopwords,pos_tags,lemmatized_review
0,"[cool, daughter, share, needle, stay, hospital...",5,1,"[very, cool, this, was, for, my, daughter, who...","[cool, daughter, share, needle, staying, hospi...","[(cool, NN), (daughter, NN), (share, NN), (nee...","[cool, daughter, share, needle, stay, hospital..."
1,"[hard, find, good, qualified, binder, cabinet,...",5,1,"[it, is, so, hard, to, find, a, good, qualifie...","[hard, find, good, qualified, binder, cabinet,...","[(hard, JJ), (find, VBP), (good, JJ), (qualifi...","[hard, find, good, qualified, binder, cabinet,..."
2,"[love, rhinestone, badge, holder, get, many, c...",4,1,"[i, love, my, rhinestone, badge, holder, i, ge...","[love, rhinestone, badge, holder, get, many, c...","[(love, VB), (rhinestone, NN), (badge, NN), (h...","[love, rhinestone, badge, holder, get, many, c..."
3,[love],5,1,"[love, it]",[love],"[(love, NN)]",[love]
4,"[use, folder, keep, student, file, love, folde...",5,1,"[i, use, these, folders, to, keep, student, fi...","[use, folders, keep, student, files, love, fol...","[(use, NN), (folders, NNS), (keep, VBP), (stud...","[use, folder, keep, student, file, love, folde..."


In [ ]:
X_text = df['lemmatized_review']
y = df['sentiment']
X_text_train, X_text_test, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42
)

In [ ]:
# creating feature extraction of bag of words
count_vectorizer = CountVectorizer()
X_train_bow = count_vectorizer.fit_transform(X_text_train)
X_test_bow = count_vectorizer.transform(X_text_test)


AttributeError: ignored

In [ ]:

X_text = df['lemmatized_review']
y = df['sentiment']
X_text_train, X_text_test, y_train, y_test = train_test_split(
    X_text, y, test_size=0.2, random_state=42
)
# creating feature extraction of tfidf
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_text_train)
X_test_tfidf = tfidf_vectorizer.transform(X_text_test)


AttributeError: ignored

# Perceptron Using Both Features

In [ ]:
# Model Training
# Perceptron using BoW and TFIDF
perceptron = Perceptron()

perceptron.fit(X_train_bow, y_train)
y_pred_perceptron = perceptron.predict(X_test_bow)
y_pred = perceptron.predict(X_test_bow)
# Calculate accuracy F1-score, precision, and recall
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Perceptron - BOW")
print(f"F1-score: {f1:.2f} Precision: {precision:.2f} Recall: {recall:.2f}")


NameError: ignored

In [ ]:
perceptron.fit(X_train_tfidf, y_train)
y_pred_perceptron = perceptron.predict(X_test_tfidf)
y_pred = perceptron.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Perceptron - TFIDF")
print(f"F1-score: {f1:.2f} Precision: {precision:.2f} Recall: {recall:.2f}")



# SVM Using Both Features

In [ ]:
linear_svc_classifier = LinearSVC()

linear_svc_classifier.fit(X_train_bow, y_train)
y_pred = linear_svc_classifier.predict(X_test_bow)

f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("SVM - BOW")
print(f"F1-score: {f1:.2f} Precision: {precision:.2f} Recall: {recall:.2f}")



In [ ]:
linear_svc_classifier.fit(X_train_tfidf, y_train)

# Make predictions on the test data
y_pred = linear_svc_classifier.predict(X_test_tfidf)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
# Calculate F1-score, precision, and recall
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("SVM - TFIDF")
print(f"F1-score: {f1:.2f} Precision: {precision:.2f} Recall: {recall:.2f}")



# Logistic Regression Using Both Features

In [ ]:
logistic_regression = LogisticRegression(penalty='l2', max_iter=1000)

logistic_regression.fit(X_train_bow, y_train)
y_pred = logistic_regression.predict(X_test_bow)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("logistic regression - BOW")
print(f"F1-score: {f1:.2f} Precision: {precision:.2f} Recall: {recall:.2f}")



In [ ]:
logistic_regression.fit(X_train_tfidf, y_train)
y_pred = logistic_regression.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("logistic regression - TFIDF")
print(f"F1-score: {f1:.2f} Precision: {precision:.2f} Recall: {recall:.2f}")



# Naive Bayes Using Both Features

In [ ]:
multinomial_nb = MultinomialNB()

#range hyperparameters to search alpha for hyperparametre tuning
param_grid = {'alpha': [0.05, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5]}

# GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(multinomial_nb, param_grid, cv=5, scoring='f1')

grid_search.fit(X_train_bow, y_train)


# best hyperparameter
best_alpha = grid_search.best_params_['alpha']

multinomial_nb_best = MultinomialNB(alpha=best_alpha)
multinomial_nb_best.fit(X_train_bow, y_train)
y_pred = multinomial_nb_best.predict(X_test_bow)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes - BOW")
print(f"F1-score: {f1:.2f} Precision: {precision:.2f} Recall: {recall:.2f}")



In [ ]:
grid_search.fit(X_train_tfidf, y_train)

#best hyperparameter
best_alpha = grid_search.best_params_['alpha']

multinomial_nb_best = MultinomialNB(alpha=best_alpha)


multinomial_nb_best.fit(X_train_tfidf, y_train)
y_pred = multinomial_nb_best.predict(X_test_tfidf)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes - TFIDF")
print(f"F1-score: {f1:.2f} Precision: {precision:.2f} Recall: {recall:.2f}")
